In [1]:
%pylab inline

import matplotlib.pyplot as plt
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [6]:
import glob

In [2]:
def write_dag_job(n, La, Lb, nu, q, ka2, kb2, ka3, kb3, ka4, kb4):
    """
    Parameters
    ----------
    n : int
        Job number starting at 1 (not 0)
        
    Returns
    -------
    str : String in dagman file to submit a job to condor.
    """
    
    string = 'JOB '+str(n)+' eobrun2.sub\n'+\
    'VARS '+str(n)+' La="'+str(La)+'"\n'+\
    'VARS '+str(n)+' Lb="'+str(Lb)+'"\n'+\
    'VARS '+str(n)+' nu="'+str(nu)+'"\n'+\
    'VARS '+str(n)+' q="'+str(q)+'"\n'+\
    'VARS '+str(n)+' ka2="'+str(ka2)+'"\n'+\
    'VARS '+str(n)+' kb2="'+str(kb2)+'"\n'+\
    'VARS '+str(n)+' ka3="'+str(ka3)+'"\n'+\
    'VARS '+str(n)+' kb3="'+str(kb3)+'"\n'+\
    'VARS '+str(n)+' ka4="'+str(ka4)+'"\n'+\
    'VARS '+str(n)+' kb4="'+str(kb4)+'"'
    
    return string



def write_dag_file(paramfile, dagfile):
    """Submit a large set of jobs to condor by creating a dag file.
    """
    params = np.loadtxt(paramfile)
    Nparams = len(params)
    
    # Write the individual DAG job strings
    runs = []
    for n in range(Nparams):
        #for n in range(1, 10):
        La, Lb, nu, q, ka2, kb2, ka3, kb3, ka4, kb4 = params[n]
        
        string = write_dag_job(n+1, La, Lb, nu, q, ka2, kb2, ka3, kb3, ka4, kb4)
        runs.append(string)

    dag_string = '\n'.join(runs)
    
    # Write to file
    f = open(dagfile,'w')
    f.write(dag_string)
    f.close()

In [3]:
# paramfile = 'parspace_smallq_cheb_16.out'
# dagfile = 'parspace_smallq_cheb_16.dag'
# write_dag_file(paramfile, dagfile)

In [3]:
# paramfile = 'parspace_rand_500.out'
# dagfile = 'parspace_rand_500.dag'
# write_dag_file(paramfile, dagfile)

In [4]:
paramfile = 'parspace_rand_10000.out'
dagfile = 'parspace_rand_10000.dag'
write_dag_file(paramfile, dagfile)

In [3]:
def write_rescue_dag_file(paramfile, dagfile, nfailed):
    """Submit a large set of jobs to condor by creating a dag file.
    """
    params = np.loadtxt(paramfile)
    Nparams = len(params)
    
    # Write the individual DAG job strings
    runs = []
    for n in nfailed:
        #for n in range(1, 10):
        La, Lb, nu, q, ka2, kb2, ka3, kb3, ka4, kb4 = params[n]
        
        string = write_dag_job(n+1, La, Lb, nu, q, ka2, kb2, ka3, kb3, ka4, kb4)
        runs.append(string)

    dag_string = '\n'.join(runs)
    
    # Write to file
    f = open(dagfile,'w')
    f.write(dag_string)
    f.close()

# Find jobs that failed

In [14]:
# training_set_dir = '/home/bdlackey/cbcrom/training_set_cheb_16'
# paramfile = 'parspace_smallq_cheb_16.out'

training_set_dir = '/home/bdlackey/cbcrom/training_set_test'
paramfile = 'parspace_rand_10000.out'

In [15]:
files_succeeded = glob.glob(training_set_dir+'/*.gz')
Nsucceeded = len(files_succeeded)
print Nsucceeded

9947


In [16]:
params_succeeded = []
for filename in files_succeeded:
    params = map(float, filename.split('/')[-1].split('.')[0].replace('p', '.').split('_')[-3:])
    params_succeeded.append(params)

In [17]:
# Find the files that failed
params = np.loadtxt(paramfile)
Nparams = len(params)

In [18]:
def job_failed(n, params, params_succeeded, tol=1.0e-3):
    """Returns True of the job failed.
    """
    La, Lb, nu, q, ka2, kb2, ka3, kb3, ka4, kb4 = params[n]
    
    # Cycle through the list of succeeded jobs
    for m in range(len(params_succeeded)):
        q_suc, La_suc, Lb_suc = params_succeeded[m]
        if np.abs(q_suc/q-1.0)<tol and np.abs(La_suc/La-1.0)<tol and np.abs(Lb_suc/Lb-1.0)<tol:
            # params is in list of jobs that succeeded
            return False
    
    # params is not in list of jobs that succeeded
    print q, La, Lb
    return True

In [19]:
failed = [n for n in range(Nparams) if job_failed(n, params, params_succeeded)]
Nfailed = len(failed)

1.52183729296 514.66716689 2753.72729579
1.00777934106 689.105649281 2431.63656819
1.07708526466 836.476358756 3172.77044709
1.38462365419 1728.72188689 1899.85098488
1.21997624241 683.175619457 1054.2379711
1.48282464183 900.639625159 3397.50106154
1.60578042918 3184.45804582 890.45475998
1.02557563252 650.310188722 859.875869535
1.87463137559 788.934768409 198.465092767
1.12612622115 2861.26425351 4525.85424228
1.04724050537 502.128382564 2069.16324053
1.67220069141 229.41688193 363.173747075
1.66464905544 3056.02567104 1367.27078038
1.20144896089 2274.70602516 57.7255604585
1.17671524659 1503.4175258 3794.34887314
1.45178443819 3585.43669367 687.897682093
1.18378168515 2012.59055086 3524.59761547
1.34438750316 1370.54045281 495.160404175
1.10956140904 4766.26938054 1171.72937893
1.40284406003 2957.97844604 2732.72234309
1.31011411866 4362.20986368 4600.62876904
1.96509315372 2198.65854039 3112.31079786
1.14962665992 3251.9101343 2508.04695572
1.34653502631 277.877747989 3756.0791739

In [20]:
print failed
#print np.array([params[n] for n in failed])
print Nsucceeded, Nfailed, Nsucceeded+Nfailed

[60, 64, 78, 87, 97, 444, 2515, 2552, 4322, 4996, 5270, 5375, 5728, 6056, 6583, 7020, 7021, 7436, 7748, 7833, 7974, 8246, 8303, 8412, 8761, 8797, 9276, 9291, 9575, 9597, 9608, 9664, 9696, 9717, 9734, 9737, 9739, 9741, 9770, 9772, 9773, 9775, 9783, 9786, 9787, 9790, 9807, 9808, 9811, 9817, 9821, 9829, 9832]
9947 53 10000


In [21]:
dagfile = 'parspace_rand_10000_retry.dag'
write_rescue_dag_file(paramfile, dagfile, failed)